Data Prep and Imports

In [2]:
import pandas as pd
import numpy as np

leak_data_raw =  pd.read_csv("Gasleak Data Sets/sensor_readings.csv")
leak_data_raw.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]

weather_df = pd.read_csv("Gasleak Data Sets/weather_data.csv")
columns_to_average = ['Barometric_Pressure', 'Humidity', 'Temperature', 'Wind_Direction', 'Wind_Speed']
for col in columns_to_average:
    weather_df[col] = weather_df.groupby('timestamp')[col].transform('mean')
weather_df = weather_df.drop_duplicates()
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9
weather_df = weather_df.rename(columns={'timestamp': 'tTime'})

leak_rate_df = pd.read_csv("Gasleak Data Sets/leak_locations_and_rate.csv")
leak_rate_df =  leak_rate_df[['LeakRate', 'tStart', 'tEnd', 'Duration']]
leak_rate_df['LeakRate'] = leak_rate_df.groupby('tStart')['LeakRate'].transform('mean')
leak_rate_df = leak_rate_df.drop_duplicates()


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_66199/2550901138.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp']).astype(int) // 10**9


In [3]:
leak_data_raw['sensor_avg'] = 0
for i in range(len(leak_data_raw)):
    leak_data_raw.iloc[i,-1] =  leak_data_raw.iloc[i, 2:-1].mean()


/var/folders/qt/r87857tx74n48x9wj7f9wljw0000gn/T/ipykernel_66199/4017535932.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1050.1875' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  leak_data_raw.iloc[i,-1] =  leak_data_raw.iloc[i, 2:-1].mean()


Merging Data Sets

In [4]:
# # Leak and weather data merge
# leak_data_raw['tTime'] = leak_data_raw['Time']
# leak_data_raw['tTime'] = pd.to_datetime(leak_data_raw['tTime'], unit='s')
# weather_df['tTime'] = pd.to_datetime(weather_df['tTime'], unit='s')

# leak_data_raw['tTime'] = leak_data_raw['tTime'].dt.floor('min').dt.round('min')
# df = pd.merge(leak_data_raw, weather_df, on='tTime', how='left')
# df = df.drop(['temp', 'tTime'], axis=1)

In [ ]:
# Normalizing leak intensity
# df['leak_amount'] = 0
# for index, row in leak_rate_df.iterrows():
#     in_range = df['Time'].between(row['tStart'], row['tEnd'], inclusive='both')
#     df.loc[in_range, 'leak_amount'] = np.array((row['LeakRate'] * (row['Duration'])), dtype=np.int32)
# df['leak_amount_norm'] = (df['leak_amount'] - df['leak_amount'].min()) / (df['leak_amount'].max() - df['leak_amount'].min())


# # Final prep
# df = df.drop(['leak_amount'],axis=1)
# df = df.astype(dtype='float32')


In [5]:
df = leak_data_raw.drop(['temp','Time'], axis=1)
# df = df.astype(dtype='float32')


Train Model

In [20]:
# Imports
import torch

import lightning as L
from torch.utils.data import TensorDataset, DataLoader
import model as m


model = m.LightningLSTM()

In [21]:
sensor1_df_normalized = (df['Sensor 1'] - df['Sensor 1'].min()) / (df['Sensor 1'].max() - df['Sensor 1'].min())
sensor1_df_normalized = sensor1_df_normalized.astype(dtype='float32')

In [22]:
sensor1_df_normalized.iloc[0:60, ]

0     0.315134
1     0.318008
2     0.307471
3     0.318966
4     0.322318
5     0.325671
6     0.347701
7     0.342433
8     0.337165
9     0.321839
10    0.330460
11    0.330460
12    0.330460
13    0.311303
14    0.301245
15    0.291188
16    0.267241
17    0.278257
18    0.289272
19    0.285441
20    0.281609
21    0.267241
22    0.268199
23    0.274904
24    0.281609
25    0.288314
26    0.295019
27    0.301724
28    0.297893
29    0.326628
30    0.329502
31    0.332375
32    0.338123
33    0.338123
34    0.319923
35    0.306513
36    0.293103
37    0.289272
38    0.280651
39    0.280651
40    0.280651
41    0.294061
42    0.297893
43    0.301724
44    0.318966
45    0.313218
46    0.307471
47    0.295977
48    0.274425
49    0.252874
50    0.226054
51    0.225096
52    0.222222
53    0.219349
54    0.219828
55    0.220307
56    0.222222
57    0.230843
58    0.232759
59    0.234674
Name: Sensor 1, dtype: float32

In [23]:
window_size = 60

# Create a list of input sequences
inputs = [sensor1_df_normalized.iloc[i:i+window_size].values for i in range(len(sensor1_df_normalized) - window_size)]

# Create a list of corresponding labels
labels = sensor1_df_normalized.iloc[window_size:].values

inputs = torch.tensor(np.array(inputs), dtype=torch.float32) # convert to numpy
labels = torch.tensor(np.array(labels), dtype=torch.float32)

dataset =  TensorDataset(inputs, labels) #fix dimension
dataloader = DataLoader(dataset)


In [10]:
trainer = L.Trainer(max_epochs=5)
trainer.fit(model,train_dataloaders=dataloader)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 16    
------------------------------
16        Trainable params
0         Non-trainable params
16        Total params
0.000     Total estimated model params size (MB)
/opt/homebrew/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0:   1%|          | 471/83353 [00:03<09:12, 149.88it/s, v_num=24]

/opt/homebrew/lib/python3.9/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [24]:
test_data = pd.read_csv("Gasleak Data Sets/validation_files/sensor_readings.csv")
test_data.columns = ["temp","Time","Sensor 1", "Sensor 2", "Sensor 3", "Sensor 4", "Sensor 5", "Sensor 6", "Sensor 7", "Sensor 8", "Sensor 9", "Sensor 10", "Sensor 11", "Sensor 12", "Sensor 13", "Sensor 14", "Sensor 15", "Sensor 16", "Sensor 17", "Sensor 18", "Sensor 19", "Sensor 20", "Sensor 21", "Sensor 22", "Sensor 23", "Sensor 24"]
sensor1_test_normalized = (test_data['Sensor 1'] - test_data['Sensor 1'].min()) / (test_data['Sensor 1'].max() - test_data['Sensor 1'].min())
sensor1_test_normalized = sensor1_test_normalized.astype(dtype='float32')

In [25]:
# Set the window size
window_size = 60

# Create a list of input sequences
inputs = [sensor1_test_normalized.iloc[i:i+window_size].values for i in range(len(sensor1_test_normalized) - window_size)]

# Create a list of corresponding labels
labels = sensor1_test_normalized.iloc[window_size:].values

# # Convert the lists to pandas objects
prediction_df = pd.concat([pd.DataFrame(inputs), pd.Series(labels)], axis=1)

test_inputs = torch.tensor(np.array(inputs), dtype=torch.float32) # convert to numpy
test_labels = torch.tensor(np.array(labels), dtype=torch.float32)

In [26]:
prediction_df.shape

(83728, 61)

In [28]:
prediction_normalized = model(test_inputs[0])
prediction = prediction_normalized * (test_data['Sensor 1'].max() - test_data['Sensor 1'].min()) + test_data['Sensor 1'].min()

# predictions = prediction_df.iloc[57:, :-1].apply(lambda x: model(torch.tensor(x, dtype=torch.float32)), axis=1)
prediction

tensor([484.8488], grad_fn=<AddBackward0>)

In [25]:
# Assuming prediction_df is a pandas DataFrame and model is a PyTorch model

# Convert the input data to a PyTorch tensor
inputs = torch.tensor(prediction_df.iloc[:, :-1].values, dtype=torch.float32)

# Create a list to store the predictions
predictions = []

# Iterate over each input sequence
for input_seq in inputs:
    # Reshape the input sequence to (1, 60)
    input_seq = input_seq.view(1, 60)
    
    # Pass the input sequence through the model
    prediction = model(input_seq)
    
    # Append the prediction to the list
    predictions.append(prediction)

# Stack the predictions into a single tensor
predictions = torch.cat(predictions, dim=0)

RuntimeError: shape '[1, 1]' is invalid for input of size 60

In [ ]:
# path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path
# trainer = L.Trainer(max_epochs=4)
# trainer.fit(model,train_dataloaders=dataloader, ckpt_path=path_to_best_checkpoint)

In [ ]:
# print(model(torch.tensor([848.0,850.0,855.0,854.0,855.0,853.0,852.5,852.0,859.0,855.0])).detach())